In [1]:
import pickle
import random

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras import backend as K
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import *
from tensorflow.python.keras.optimizers import *
from tensorflow.python.keras.callbacks import ModelCheckpoint

gpu_options = tf.GPUOptions(allow_growth=True, per_process_gpu_memory_fraction=0.1)

print(tf.__version__)
print(keras.__version__)

1.10.0
2.2.4


Using TensorFlow backend.


In [2]:
with open("data.txt", "rb") as fp:   # Unpickling
    data = pickle.load(fp)
random.shuffle(data)
border = int(0.8 * len(data))
dataTrain = np.array(data[:border])
dataVal = np.array(data[border:])

xTrain, yTrain = dataTrain[:,0], dataTrain[:,1]
xVal, yVal = dataVal[:,0], dataVal[:,1]
#xTest, yTest = dataTest[:,0], dataTest[:,1]

xTrain2 = []
for element in xTrain:
    new_element = np.empty((15,15,3))
    for i in range(3):
        new_element[:,:,i] = element[i,:,:]
    xTrain2.append(new_element)
xTrain2 = np.array(xTrain2)

xVal2 = []
for element in xVal:
    new_element = np.empty((15,15,3))
    for i in range(3):
        new_element[:,:,i] = element[i,:,:]
    xVal2.append(new_element)
xVal2 = np.array(xVal2)

yTrain = keras.utils.to_categorical(yTrain, 225)
yVal = keras.utils.to_categorical(yVal, 225)
#yTest = keras.utils.to_categorical(yTest, 225)

In [3]:
print(xTrain2.shape)
print(yTrain.shape)
print(xVal2.shape)
print(yVal.shape)

(912488, 15, 15, 3)
(912488, 225)
(228123, 15, 15, 3)
(228123, 225)


In [4]:
def make_model():
    model = Sequential()
    model.add(Conv2D(32, 3, input_shape=(15, 15, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, 3, padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(128, 3, padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, 3, padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(32, 3, padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(16, 3, padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(1, 3, padding='same'))
    model.add(Activation('relu'))
    
    model.add(Flatten())
    
    model.add(Dense(225))
    model.add(Activation("softmax"))

    model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
from tensorflow.python.keras.callbacks import CSVLogger
logger = CSVLogger('log.csv', append=True, separator=';')

INIT_LR = 8e-3  # initial learning rate
BATCH_SIZE = 1024
EPOCHS = 500
#K.clear_session()  # clear default graph
with tf.device('/device:GPU:3'):
    model = load_model('night_model.hdf5')
    #model = make_model()  # define our model
    model_checkpoint = ModelCheckpoint('night_model.hdf5', monitor='loss', verbose=1, save_best_only=True)

    history = model.fit(
        xTrain2, yTrain,  # prepared data
        batch_size=BATCH_SIZE,
        callbacks=[model_checkpoint, logger],
        epochs=EPOCHS,
        validation_data=(xVal2, yVal),
        shuffle=True,
    )

Train on 912488 samples, validate on 228123 samples
Epoch 1/500
912488/912488 [==============================] - 160s 175us/step - loss: 3.7741 - acc: 0.1033 - val_loss: 3.7407 - val_acc: 0.1065

Epoch 00001: loss improved from inf to 3.77413, saving model to night_model.hdf5
Epoch 2/500
912488/912488 [==============================] - 162s 178us/step - loss: 3.7027 - acc: 0.1126 - val_loss: 3.6831 - val_acc: 0.1172

Epoch 00002: loss improved from 3.77413 to 3.70265, saving model to night_model.hdf5
Epoch 3/500
912488/912488 [==============================] - 162s 177us/step - loss: 3.6217 - acc: 0.1295 - val_loss: 3.5804 - val_acc: 0.1393

Epoch 00003: loss improved from 3.70265 to 3.62172, saving model to night_model.hdf5
Epoch 4/500
912488/912488 [==============================] - 162s 178us/step - loss: 3.5054 - acc: 0.1528 - val_loss: 3.4459 - val_acc: 0.1626

Epoch 00004: loss improved from 3.62172 to 3.50542, saving model to night_model.hdf5
Epoch 5/500
912488/912488 [=========


Epoch 00036: loss improved from 1.88528 to 1.87880, saving model to night_model.hdf5
Epoch 37/500
912488/912488 [==============================] - 162s 177us/step - loss: 1.8703 - acc: 0.4669 - val_loss: 1.9311 - val_acc: 0.4539

Epoch 00037: loss improved from 1.87880 to 1.87025, saving model to night_model.hdf5
Epoch 38/500
912488/912488 [==============================] - 162s 177us/step - loss: 1.8642 - acc: 0.4675 - val_loss: 1.9211 - val_acc: 0.4548

Epoch 00038: loss improved from 1.87025 to 1.86421, saving model to night_model.hdf5
Epoch 39/500
912488/912488 [==============================] - 162s 177us/step - loss: 1.8572 - acc: 0.4690 - val_loss: 1.9346 - val_acc: 0.4536

Epoch 00039: loss improved from 1.86421 to 1.85719, saving model to night_model.hdf5
Epoch 40/500
912488/912488 [==============================] - 162s 177us/step - loss: 1.8518 - acc: 0.4700 - val_loss: 1.9153 - val_acc: 0.4568

Epoch 00040: loss improved from 1.85719 to 1.85178, saving model to night_model

912488/912488 [==============================] - 161s 177us/step - loss: 1.7297 - acc: 0.4944 - val_loss: 1.8358 - val_acc: 0.4688

Epoch 00072: loss improved from 1.73231 to 1.72970, saving model to night_model.hdf5
Epoch 73/500
912488/912488 [==============================] - 161s 177us/step - loss: 1.7270 - acc: 0.4953 - val_loss: 1.8308 - val_acc: 0.4707

Epoch 00073: loss improved from 1.72970 to 1.72703, saving model to night_model.hdf5
Epoch 74/500
912488/912488 [==============================] - 161s 177us/step - loss: 1.7249 - acc: 0.4957 - val_loss: 1.8330 - val_acc: 0.4710

Epoch 00074: loss improved from 1.72703 to 1.72486, saving model to night_model.hdf5
Epoch 75/500
912488/912488 [==============================] - 161s 177us/step - loss: 1.7223 - acc: 0.4961 - val_loss: 1.8253 - val_acc: 0.4715

Epoch 00075: loss improved from 1.72486 to 1.72232, saving model to night_model.hdf5
Epoch 76/500
912488/912488 [==============================] - 161s 177us/step - loss: 1.7199 

912488/912488 [==============================] - 160s 176us/step - loss: 1.6594 - acc: 0.5102 - val_loss: 1.8025 - val_acc: 0.4753

Epoch 00108: loss improved from 1.66089 to 1.65935, saving model to night_model.hdf5
Epoch 109/500
912488/912488 [==============================] - 160s 176us/step - loss: 1.6577 - acc: 0.5098 - val_loss: 1.8016 - val_acc: 0.4733

Epoch 00109: loss improved from 1.65935 to 1.65775, saving model to night_model.hdf5
Epoch 110/500
912488/912488 [==============================] - 161s 176us/step - loss: 1.6572 - acc: 0.5105 - val_loss: 1.8045 - val_acc: 0.4730

Epoch 00110: loss improved from 1.65775 to 1.65722, saving model to night_model.hdf5
Epoch 111/500
912488/912488 [==============================] - 161s 176us/step - loss: 1.6552 - acc: 0.5106 - val_loss: 1.8114 - val_acc: 0.4773

Epoch 00111: loss improved from 1.65722 to 1.65522, saving model to night_model.hdf5
Epoch 112/500
912488/912488 [==============================] - 160s 176us/step - loss: 1.6

KeyboardInterrupt: 

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(loss)
plt.plot(val_loss)
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
# make test predictions
yTestPred = model.predict_proba(xTest2)
win = 0
for i in range(100):
    a = np.argmax(yTestPred[i])
    b = yTest[i]
    print(a, b)
    print('----')
    if a == b:
        win += 1
print(win)
#y_pred_test_classes = np.argmax(yTestPred, axis=1)
#y_pred_test_max_probas = np.max(yTestPred, axis=1)